<a href="https://colab.research.google.com/github/LucasDatilioCarderelli/Maratona_BehindTheCode_IBM21/blob/main/MBTC2021_df1_TotalBan_pt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MBTC2021_DF1-TotalBan

## Imports

### installs

In [ ]:
# !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
!pip install tpot

     |████████████████████████████████| 87 kB 3.1 MB/s 
     |████████████████████████████████| 173.5 MB 33 kB/s 
     |████████████████████████████████| 160 kB 62.7 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=e0726579d50aa84e8e48f8a07bf35871bd2e03cb130f0e0303b2da2899416d02
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from tpot import TPOTClassifier
from xgboost import XGBClassifier

### Get Data

In [ ]:
accounts_url = "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/ACCOUNTS.csv"
demograph_url = "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/DEMOGRAPHICS.csv"
loans_url = "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/LOANS.csv"

df_accounts = pd.read_csv(accounts_url)
df_demograph = pd.read_csv(demograph_url)
df_loans = pd.read_csv(loans_url)

Variables

    "ID", # Número de identificação do cliente
    "CHECKING_BALANCE", # Saldo em conta corrente do cliente
    "CREDIT_HISTORY", # Como está a situação de crédito passada do cliente
    "EXISTING_SAVINGS", # Saldo de conta poupança
    "EXISTING_CREDITS_COUNT", # Número de empréstimos que o cliente já tem
    "SEX", # Sexo
    "AGE", # Idade
    "JOB_TYPE", # Tipo de emprego: 0 - desempregado, 1 - Não qualificado, 2 - Autônomo, 3 - Qualificado
    "DEPENDENTS", # Número de pessoas com acesso à conta
    "TELEPHONE", # Se o cliente tem telefone cadastrado ou não
    "FOREIGN_WORKER", # Se o cliente trabalha num país externo ao do banco ou não
    "EMPLOYMENT_DURATION", # Quantos anos o cliente está no último emprego
    "PROPERTY", # Se o cliente possui alguma propriedade em nome
    "HOUSING", # Tem casa própria ou não
    "CURRENT_RESIDENCE_DURATION", # Anos em que o cliente está vivendo na última casa
    "PAYMENT_TERM", # Número de dias que o cliente tem para pagar o empréstimo
    "INSTALLMENT_PLANS", # Plano de financiamento, podendo ser do banco, externo, ou nenhum
    "INSTALLMENT_PERCENT", # Em quantas parcelas o empréstimo deve ser pago
    "LOAN_PURPOSE", # Motivação do empréstimo
    "LOAN_AMOUNT", # Valor do empréstimo
    "OTHERS_ON_LOAN" # Se existe um fiador ou outro aplicante para o empréstimo

Target

    "ALLOW"

In [ ]:
df_accounts

,ID,CHECKING_BALANCE,CREDIT_HISTORY,EXISTING_SAVINGS,EXISTING_CREDITS_COUNT
0,291655,-566.53,ALL_CREDITS_PAID_BACK,36.42,1.0
1,657079,-1673.16,CREDITS_PAID_TO_DATE,22.9,1.0
2,256050,NO_CHECKING,CREDITS_PAID_TO_DATE,450.35,2.0
3,1039151,-1471.03,PRIOR_PAYMENTS_DELAYED,805.71,2.0
4,1002660,NO_CHECKING,CREDITS_PAID_TO_DATE,295.03,1.0
...,...,...,...,...,...
2472,636739,163.82,CREDITS_PAID_TO_DATE,472.11,1.0
2473,919733,14324.23,PRIOR_PAYMENTS_DELAYED,18652.44,1.0
2474,115839,130.13,PRIOR_PAYMENTS_DELAYED,726.78,2.0
2475,1128684,NO_CHECKING,PRIOR_PAYMENTS_DELAYED,UNKNOWN,1.0


In [ ]:
df_demograph

,ID,SEX,AGE,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,EMPLOYMENT_DURATION,PROPERTY,HOUSING,CURRENT_RESIDENCE_DURATION
0,24252,M,47.0,3.0,1.0,0.0,1.0,14.0,SAVINGS_INSURANCE,OWN,3.0
1,1002018,M,19.0,3.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,RENT,2.0
2,1240455,F,50.0,2.0,1.0,1.0,1.0,7.0,UNKNOWN,OWN,3.0
3,107799,M,35.0,3.0,1.0,0.0,1.0,0.0,REAL_ESTATE,OWN,2.0
4,184883,M,40.0,1.0,1.0,0.0,1.0,9.0,CAR_OTHER,OWN,4.0
...,...,...,...,...,...,...,...,...,...,...,...
3931,636739,F,41.0,0.0,1.0,0.0,1.0,7.0,CAR_OTHER,OWN,3.0
3932,919733,M,46.0,3.0,1.0,1.0,1.0,11.0,SAVINGS_INSURANCE,OWN,3.0
3933,115839,M,28.0,3.0,1.0,1.0,1.0,7.0,CAR_OTHER,RENT,1.0
3934,1128684,M,46.0,0.0,1.0,0.0,1.0,7.0,CAR_OTHER,RENT,3.0


In [ ]:
df_loans

,ID,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN,ALLOW
0,610339,713.0,STORES,3.0,CAR_USED,3103.0,NONE,1
1,109167,341.0,STORES,1.0,CAR_NEW,716.0,NONE,1
2,823322,868.0,NONE,4.0,APPLIANCES,4333.0,NONE,1
3,322224,310.0,STORES,2.0,FURNITURE,250.0,NONE,1
4,912693,620.0,NONE,2.0,CAR_USED,2454.0,NONE,0
...,...,...,...,...,...,...,...,...
3994,636739,450.0,BANK,3.0,CAR_NEW,3007.0,CO-APPLICANT,0
3995,919733,510.0,NONE,4.0,VACATION,4191.0,GUARANTOR,1
3996,115839,124.0,STORES,1.0,FURNITURE,250.0,NONE,1
3997,1128684,961.0,BANK,3.0,BUSINESS,5172.0,NONE,1


### Merging

In [ ]:
df = df_accounts.sort_values("ID").set_index("ID").join(
    [df_demograph.set_index("ID"), df_loans.set_index("ID")],
    how="outer")
df

,CHECKING_BALANCE,CREDIT_HISTORY,EXISTING_SAVINGS,EXISTING_CREDITS_COUNT,SEX,AGE,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,EMPLOYMENT_DURATION,PROPERTY,HOUSING,CURRENT_RESIDENCE_DURATION,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN,ALLOW
ID,,,,,,,,,,,,,,,,,,,,,
98,32.83,CREDITS_PAID_TO_DATE,476.7,1.0,F,32.0,3.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,OWN,3.0,961.0,NONE,3.0,OTHER,1889.0,NONE,1
317,-150.05,CREDITS_PAID_TO_DATE,49.04,2.0,F,37.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,2.0,540.0,STORES,2.0,CAR_NEW,462.0,NONE,1
407,-185.04,PRIOR_PAYMENTS_DELAYED,49.65,2.0,M,28.0,3.0,1.0,1.0,0.0,3.0,REAL_ESTATE,OWN,3.0,465.0,NONE,2.0,FURNITURE,250.0,NONE,1
2375,NaN,NaN,NaN,NaN,F,36.0,3.0,1.0,0.0,1.0,6.0,CAR_OTHER,OWN,1.0,480.0,NONE,3.0,VACATION,3109.0,NONE,1
2731,NaN,NaN,NaN,NaN,F,22.0,2.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,OWN,3.0,330.0,NONE,3.0,CAR_NEW,4553.0,NONE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319170,9641.84,CREDITS_PAID_TO_DATE,72.52,1.0,M,40.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,4.0,810.0,NONE,3.0,FURNITURE,4650.0,NONE,1
1319397,0,PRIOR_PAYMENTS_DELAYED,4580.14,1.0,M,32.0,0.0,1.0,0.0,1.0,7.0,CAR_OTHER,OWN,3.0,330.0,BANK,3.0,FURNITURE,250.0,NONE,1
1319835,NaN,NaN,NaN,NaN,M,54.0,1.0,2.0,1.0,1.0,11.0,UNKNOWN,OWN,5.0,992.0,STORES,5.0,APPLIANCES,6536.0,CO-APPLICANT,0


In [ ]:
# profile = ProfileReport(df)
# profile.to_file("your_report.html")

## Droping NA

In [ ]:
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2237 entries, 98 to 1319964
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CHECKING_BALANCE            2237 non-null   object 
 1   CREDIT_HISTORY              2237 non-null   object 
 2   EXISTING_SAVINGS            2237 non-null   object 
 3   EXISTING_CREDITS_COUNT      2237 non-null   float64
 4   SEX                         2237 non-null   object 
 5   AGE                         2237 non-null   float64
 6   JOB_TYPE                    2237 non-null   float64
 7   DEPENDENTS                  2237 non-null   float64
 8   TELEPHONE                   2237 non-null   float64
 9   FOREIGN_WORKER              2237 non-null   float64
 10  EMPLOYMENT_DURATION         2237 non-null   float64
 11  PROPERTY                    2237 non-null   object 
 12  HOUSING                     2237 non-null   object 
 13  CURRENT_RESIDENCE_DURATION  2

## Modeling

In [ ]:
df

,CHECKING_BALANCE,CREDIT_HISTORY,EXISTING_SAVINGS,EXISTING_CREDITS_COUNT,SEX,AGE,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,EMPLOYMENT_DURATION,PROPERTY,HOUSING,CURRENT_RESIDENCE_DURATION,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN,ALLOW
ID,,,,,,,,,,,,,,,,,,,,,
98,32.83,CREDITS_PAID_TO_DATE,476.7,1.0,F,32.0,3.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,OWN,3.0,961.0,NONE,3.0,OTHER,1889.0,NONE,1
317,-150.05,CREDITS_PAID_TO_DATE,49.04,2.0,F,37.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,2.0,540.0,STORES,2.0,CAR_NEW,462.0,NONE,1
407,-185.04,PRIOR_PAYMENTS_DELAYED,49.65,2.0,M,28.0,3.0,1.0,1.0,0.0,3.0,REAL_ESTATE,OWN,3.0,465.0,NONE,2.0,FURNITURE,250.0,NONE,1
2893,NO_CHECKING,OUTSTANDING_CREDIT,695.07,2.0,M,49.0,3.0,2.0,1.0,1.0,10.0,UNKNOWN,FREE,4.0,1050.0,NONE,5.0,APPLIANCES,7138.0,CO-APPLICANT,0
3412,-51.41,ALL_CREDITS_PAID_BACK,71.68,1.0,F,19.0,2.0,1.0,0.0,1.0,0.0,REAL_ESTATE,RENT,2.0,155.0,NONE,2.0,CAR_NEW,1523.0,NONE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317323,-1814.54,NO_CREDITS,83.97,1.0,F,32.0,3.0,1.0,0.0,1.0,3.0,SAVINGS_INSURANCE,OWN,2.0,660.0,NONE,3.0,FURNITURE,1822.0,NONE,1
1318378,128.65,CREDITS_PAID_TO_DATE,391.93,2.0,M,43.0,3.0,1.0,0.0,1.0,3.0,CAR_OTHER,OWN,3.0,450.0,NONE,3.0,CAR_USED,4097.0,NONE,1
1319170,9641.84,CREDITS_PAID_TO_DATE,72.52,1.0,M,40.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,4.0,810.0,NONE,3.0,FURNITURE,4650.0,NONE,1


### Spliting

In [ ]:
features = ['PAYMENT_TERM', 'INSTALLMENT_PERCENT', 'LOAN_AMOUNT']
target = ['ALLOW']

X = df[features]
y = df[target]

In [ ]:
test_pct = 0.3 # Separaremos 30% dos dados para testes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_pct, random_state=42)

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo (número de predições assertivas sobre número total de testes): {acc}")

Acurácia do modelo (número de predições assertivas sobre número total de testes): 0.6532738095238095


In [ ]:
XGB = XGBClassifier()
XGB.fit(X_train, y_train.values.ravel())
res = XGB.predict(X_test)
print(f1_score(y_test, res))

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7848383500557414


### Testings

In [ ]:
tpot = TPOTClassifier(generations=10, population_size=200, verbosity=2, 
                      scoring="f1", random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/2200 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8270104278040005

Generation 2 - Current best internal CV score: 0.8295227494349353

Generation 3 - Current best internal CV score: 0.8295227494349353

Generation 4 - Current best internal CV score: 0.8295227494349353

Generation 5 - Current best internal CV score: 0.8295227494349353

Generation 6 - Current best internal CV score: 0.8295227494349353

Generation 7 - Current best internal CV score: 0.8295227494349353

Generation 8 - Current best internal CV score: 0.8295227494349353

Generation 9 - Current best internal CV score: 0.8295227494349353

Generation 10 - Current best internal CV score: 0.8295227494349353

Best pipeline: ExtraTreesClassifier(SGDClassifier(input_matrix, alpha=0.001, eta0=0.01, fit_intercept=True, l1_ratio=0.75, learning_rate=invscaling, loss=squared_hinge, penalty=elasticnet, power_t=0.5), bootstrap=True, criterion=entropy, max_features=0.2, min_samples_leaf=12, min_samples_split=3, n_estimators=100)
0.8343558282

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
score1 = (0.8300000000000001 + 0.8261306532663317 + 0.8314833501513622 + 0.8329938900203666 + 0.8274470232088799)/5
score2 = (0.8229475766567754 + 0.8316430020283976 + 0.8256513026052104 + 0.815686274509804 + 0.8242303872889771)/5
print(f'score1: {score1}\nscore2: {score2}')

score1: 0.829610983329388
score2: 0.8240317086178329


In [ ]:
from tpot.export_utils      import set_param_recursive
from tpot.builtins          import StackingEstimator
from sklearn.pipeline       import make_pipeline, make_union
from sklearn.decomposition  import PCA
from sklearn.preprocessing  import MaxAbsScaler, RobustScaler
from sklearn.ensemble       import ExtraTreesClassifier
from sklearn.linear_model   import SGDClassifier
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.tree           import DecisionTreeClassifier

#### 3 columns

In [ ]:
# Average CV score on the training set was: 0.8295227494349353
exported_pipeline = make_pipeline(
    # StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=0.01, fit_intercept=True, l1_ratio=0.75, learning_rate="invscaling", loss="squared_hinge", penalty="elasticnet", power_t=0.5)),
    ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.2, min_samples_leaf=12, min_samples_split=3, n_estimators=100)
)
exported_pipeline.fit(X_train, y_train.values.ravel())
results = exported_pipeline.predict(X_test)
print(f1_score(y_test, results))

0.8250497017892645


In [ ]:
# Average CV score on the training set was: 0.7456869009584665
model1 = make_pipeline(
    # RobustScaler(),
    KNeighborsClassifier(n_neighbors=76, p=1, weights="uniform")
)
model1.fit(X_train, y_train.values.ravel())
results = model1.predict(X_test)
print(f1_score(y_test, results))

0.8153846153846155


In [ ]:
# Average CV score on the training set was: 0.828125594438782
model2 = make_pipeline(
    # StackingEstimator(estimator=SGDClassifier(alpha=0.0, eta0=0.01, fit_intercept=False, l1_ratio=0.5, learning_rate="invscaling", loss="log", penalty="elasticnet", power_t=10.0)),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.2, min_samples_leaf=18, min_samples_split=19, n_estimators=100)
)
model2.fit(X_train, y_train.values.ravel())
results = model2.predict(X_test)
print(f1_score(y_test, results))

0.8258706467661692


In [ ]:
# Average CV score on the training set was: 0.8265843316312672
model3 = make_pipeline(
    RobustScaler(),
    PCA(iterated_power=6, svd_solver="randomized"),
    MaxAbsScaler(),
    DecisionTreeClassifier(criterion="entropy", max_depth=2, min_samples_leaf=12, min_samples_split=12)
)
model3.fit(X_train, y_train.values.ravel())
results = model3.predict(X_test)
print(f1_score(y_test, results))

0.8231768231768232


#### More Columns

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2237 entries, 98 to 1319964
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CHECKING_BALANCE            2237 non-null   object 
 1   CREDIT_HISTORY              2237 non-null   object 
 2   EXISTING_SAVINGS            2237 non-null   object 
 3   EXISTING_CREDITS_COUNT      2237 non-null   float64
 4   SEX                         2237 non-null   object 
 5   AGE                         2237 non-null   float64
 6   JOB_TYPE                    2237 non-null   float64
 7   DEPENDENTS                  2237 non-null   float64
 8   TELEPHONE                   2237 non-null   float64
 9   FOREIGN_WORKER              2237 non-null   float64
 10  EMPLOYMENT_DURATION         2237 non-null   float64
 11  PROPERTY                    2237 non-null   object 
 12  HOUSING                     2237 non-null   object 
 13  CURRENT_RESIDENCE_DURATION  2

In [ ]:
features = ['CREDIT_HISTORY', 'JOB_TYPE', 'DEPENDENTS',
            'TELEPHONE', 'FOREIGN_WORKER', 'PROPERTY',
            'HOUSING', 'PAYMENT_TERM', 'INSTALLMENT_PERCENT', 
            'LOAN_PURPOSE', 'LOAN_AMOUNT']
target = ['ALLOW']

X = df[features]
y = df[target]

In [ ]:
X = pd.get_dummies(X, columns=X.select_dtypes("object").columns)
X

,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,PAYMENT_TERM,INSTALLMENT_PERCENT,LOAN_AMOUNT,CREDIT_HISTORY_ALL_CREDITS_PAID_BACK,CREDIT_HISTORY_CREDITS_PAID_TO_DATE,CREDIT_HISTORY_NO_CREDITS,CREDIT_HISTORY_OUTSTANDING_CREDIT,CREDIT_HISTORY_PRIOR_PAYMENTS_DELAYED,PROPERTY_CAR_OTHER,PROPERTY_REAL_ESTATE,PROPERTY_SAVINGS_INSURANCE,PROPERTY_UNKNOWN,HOUSING_FREE,HOUSING_OWN,HOUSING_RENT,LOAN_PURPOSE_APPLIANCES,LOAN_PURPOSE_BUSINESS,LOAN_PURPOSE_CAR_NEW,LOAN_PURPOSE_CAR_USED,LOAN_PURPOSE_EDUCATION,LOAN_PURPOSE_FURNITURE,LOAN_PURPOSE_OTHER,LOAN_PURPOSE_RADIO_TV,LOAN_PURPOSE_REPAIRS,LOAN_PURPOSE_RETRAINING,LOAN_PURPOSE_VACATION
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
98,3.0,1.0,0.0,1.0,961.0,3.0,1889.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
317,3.0,1.0,0.0,1.0,540.0,2.0,462.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
407,3.0,1.0,1.0,0.0,465.0,2.0,250.0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2893,3.0,2.0,1.0,1.0,1050.0,5.0,7138.0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3412,2.0,1.0,0.0,1.0,155.0,2.0,1523.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317323,3.0,1.0,0.0,1.0,660.0,3.0,1822.0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1318378,3.0,1.0,0.0,1.0,450.0,3.0,4097.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1319170,3.0,1.0,0.0,1.0,810.0,3.0,4650.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
test_pct = 0.3 # Separaremos 30% dos dados para testes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_pct, random_state=42)

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo (número de predições assertivas sobre número total de testes): {acc}")

Acurácia do modelo (número de predições assertivas sobre número total de testes): 0.7008928571428571


In [ ]:
tpot = TPOTClassifier(generations=10, population_size=100, verbosity=2, 
                      scoring="f1", random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/600 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GaussianNB(input_matrix)
0.7604562737642585


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.decomposition import FastICA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import PolynomialFeatures

# Average CV score on the training set was: 0.8349545509709028
model3 = make_pipeline(
    # FastICA(tol=0.9500000000000001),
    # PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=19, min_samples_split=20, n_estimators=100, random_state=42)
)
model3.fit(X_train, y_train)
results = model3.predict(X_test)
print(classification_report(y_test, results, digits=4))

              precision    recall  f1-score   support

           0     0.7341    0.5314    0.6165       239
           1     0.7756    0.8938    0.8305       433

    accuracy                         0.7649       672
   macro avg     0.7548    0.7126    0.7235       672
weighted avg     0.7608    0.7649    0.7544       672



/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [ ]:
XGB2 = XGBClassifier(learning_rate=0.1, max_depth=3, min_child_weight=14, 
                     n_estimators=100, subsample=0.25, random_state=42)
XGB2.fit(X_train, y_train)
predict = XGB2.predict(X_test)
print(f1_score(y_test, predict))

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8434782608695652


### Selecting Model

In [ ]:
# 0.8255
# ExtraTreesClassifier(bootstrap=True, criterion="entropy", 
#                      max_features=0.2, min_samples_leaf=12, 
#                      min_samples_split=3, n_estimators=100, random_state=42)
# 0.8260
# ExtraTreesClassifier(bootstrap=False, criterion="entropy",
#                      max_features=0.2, min_samples_leaf=18, 
#                      min_samples_split=19, n_estimators=100, random_state=42)

In [ ]:
df_loans.dropna()

,ID,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN,ALLOW
0,610339,713.0,STORES,3.0,CAR_USED,3103.0,NONE,1
1,109167,341.0,STORES,1.0,CAR_NEW,716.0,NONE,1
2,823322,868.0,NONE,4.0,APPLIANCES,4333.0,NONE,1
3,322224,310.0,STORES,2.0,FURNITURE,250.0,NONE,1
4,912693,620.0,NONE,2.0,CAR_USED,2454.0,NONE,0
...,...,...,...,...,...,...,...,...
3994,636739,450.0,BANK,3.0,CAR_NEW,3007.0,CO-APPLICANT,0
3995,919733,510.0,NONE,4.0,VACATION,4191.0,GUARANTOR,1
3996,115839,124.0,STORES,1.0,FURNITURE,250.0,NONE,1
3997,1128684,961.0,BANK,3.0,BUSINESS,5172.0,NONE,1


In [ ]:
loans = df_loans[['PAYMENT_TERM', 'INSTALLMENT_PERCENT', 'LOAN_AMOUNT', 'ALLOW']].dropna()
X = loans[['PAYMENT_TERM', 'INSTALLMENT_PERCENT', 'LOAN_AMOUNT']]
y = loans[['ALLOW']]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2752, 3), (1180, 3), (2752, 1), (1180, 1))

In [ ]:
final_model = ExtraTreesClassifier(bootstrap=True, criterion="entropy", 
                                   max_features=0.2, min_samples_leaf=12,
                                   min_samples_split=3, n_estimators=100, 
                                   random_state=42)
final_model.fit(X_train, y_train.values.ravel())
final_result = final_model.predict(X_test)
print(f1_score(y_test, final_result))

0.8349514563106797


## Changing Submission

In [ ]:
answer = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/ANSWERS.csv")
answer

,ID,CHECKING_BALANCE,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EXISTING_SAVINGS,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,ALLOW
0,1078506,25.84,465,NO_CREDITS,CAR_NEW,250,38.35,0,1,F,NONE,2,REAL_ESTATE,32,STORES,OWN,1,3,1,0,1,TO_FILL
1,947575,-439.72,120,ALL_CREDITS_PAID_BACK,CAR_NEW,250,1.62,0,2,M,CO-APPLICANT,2,REAL_ESTATE,25,STORES,RENT,1,3,1,0,1,TO_FILL
2,133043,169.45,527,PRIOR_PAYMENTS_DELAYED,RADIO_TV,2207,184.97,6,2,M,NONE,3,SAVINGS_INSURANCE,32,STORES,OWN,2,1,2,0,1,TO_FILL
3,1031383,NO_CHECKING,720,OUTSTANDING_CREDIT,CAR_USED,5064,731.03,6,4,M,NONE,2,CAR_OTHER,49,NONE,OWN,2,3,1,0,1,TO_FILL
4,491563,NO_CHECKING,420,OUTSTANDING_CREDIT,FURNITURE,3763,396.26,4,3,F,NONE,3,CAR_OTHER,46,STORES,OWN,1,1,2,1,1,TO_FILL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1023481,NO_CHECKING,1116,PRIOR_PAYMENTS_DELAYED,APPLIANCES,3960,1224.22,8,4,F,NONE,3,SAVINGS_INSURANCE,46,NONE,OWN,2,2,1,0,0,TO_FILL
996,1211101,-399.5,720,ALL_CREDITS_PAID_BACK,CAR_NEW,1834,57.44,7,3,M,NONE,1,SAVINGS_INSURANCE,27,NONE,OWN,1,3,1,0,1,TO_FILL
997,599972,158.3,558,CREDITS_PAID_TO_DATE,CAR_NEW,2348,520.17,1,3,M,CO-APPLICANT,2,SAVINGS_INSURANCE,42,STORES,OWN,1,1,1,0,1,TO_FILL
998,1146844,NO_CHECKING,1170,PRIOR_PAYMENTS_DELAYED,FURNITURE,5457,531.02,4,3,M,CO-APPLICANT,3,CAR_OTHER,42,NONE,OWN,1,3,1,1,1,TO_FILL


In [ ]:
df

,CHECKING_BALANCE,CREDIT_HISTORY,EXISTING_SAVINGS,EXISTING_CREDITS_COUNT,SEX,AGE,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,EMPLOYMENT_DURATION,PROPERTY,HOUSING,CURRENT_RESIDENCE_DURATION,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN,ALLOW
ID,,,,,,,,,,,,,,,,,,,,,
98,32.83,CREDITS_PAID_TO_DATE,476.7,1.0,F,32.0,3.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,OWN,3.0,961.0,NONE,3.0,OTHER,1889.0,NONE,1
317,-150.05,CREDITS_PAID_TO_DATE,49.04,2.0,F,37.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,2.0,540.0,STORES,2.0,CAR_NEW,462.0,NONE,1
407,-185.04,PRIOR_PAYMENTS_DELAYED,49.65,2.0,M,28.0,3.0,1.0,1.0,0.0,3.0,REAL_ESTATE,OWN,3.0,465.0,NONE,2.0,FURNITURE,250.0,NONE,1
2893,NO_CHECKING,OUTSTANDING_CREDIT,695.07,2.0,M,49.0,3.0,2.0,1.0,1.0,10.0,UNKNOWN,FREE,4.0,1050.0,NONE,5.0,APPLIANCES,7138.0,CO-APPLICANT,0
3412,-51.41,ALL_CREDITS_PAID_BACK,71.68,1.0,F,19.0,2.0,1.0,0.0,1.0,0.0,REAL_ESTATE,RENT,2.0,155.0,NONE,2.0,CAR_NEW,1523.0,NONE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317323,-1814.54,NO_CREDITS,83.97,1.0,F,32.0,3.0,1.0,0.0,1.0,3.0,SAVINGS_INSURANCE,OWN,2.0,660.0,NONE,3.0,FURNITURE,1822.0,NONE,1
1318378,128.65,CREDITS_PAID_TO_DATE,391.93,2.0,M,43.0,3.0,1.0,0.0,1.0,3.0,CAR_OTHER,OWN,3.0,450.0,NONE,3.0,CAR_USED,4097.0,NONE,1
1319170,9641.84,CREDITS_PAID_TO_DATE,72.52,1.0,M,40.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,4.0,810.0,NONE,3.0,FURNITURE,4650.0,NONE,1


In [ ]:
answer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          1000 non-null   int64 
 1   CHECKING_BALANCE            1000 non-null   object
 2   PAYMENT_TERM                1000 non-null   int64 
 3   CREDIT_HISTORY              1000 non-null   object
 4   LOAN_PURPOSE                1000 non-null   object
 5   LOAN_AMOUNT                 1000 non-null   int64 
 6   EXISTING_SAVINGS            1000 non-null   object
 7   EMPLOYMENT_DURATION         1000 non-null   int64 
 8   INSTALLMENT_PERCENT         1000 non-null   int64 
 9   SEX                         1000 non-null   object
 10  OTHERS_ON_LOAN              1000 non-null   object
 11  CURRENT_RESIDENCE_DURATION  1000 non-null   int64 
 12  PROPERTY                    1000 non-null   object
 13  AGE                         1000 non-null   int64

### Treatning

"ALLOW" High correlatin:
    
    "AGE", int64
    "TELEPHONE", int64
    "EMPLOYMENT_DURATION", int64
    "PROPERTY", object
    "CURRENT_RESIDENCE_DURATION, int64
    "PAYMENT_TERM", int64
    "INSTALLMENT_PERCENT", int64

In [ ]:
unwanted_columns = ["ALLOW", "ID", "CHECKING_BALANCE", "EXISTING_SAVINGS"]

In [ ]:
df2 = answer.drop(unwanted_columns, axis=1)
df2

,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER
0,465,NO_CREDITS,CAR_NEW,250,0,1,F,NONE,2,REAL_ESTATE,32,STORES,OWN,1,3,1,0,1
1,120,ALL_CREDITS_PAID_BACK,CAR_NEW,250,0,2,M,CO-APPLICANT,2,REAL_ESTATE,25,STORES,RENT,1,3,1,0,1
2,527,PRIOR_PAYMENTS_DELAYED,RADIO_TV,2207,6,2,M,NONE,3,SAVINGS_INSURANCE,32,STORES,OWN,2,1,2,0,1
3,720,OUTSTANDING_CREDIT,CAR_USED,5064,6,4,M,NONE,2,CAR_OTHER,49,NONE,OWN,2,3,1,0,1
4,420,OUTSTANDING_CREDIT,FURNITURE,3763,4,3,F,NONE,3,CAR_OTHER,46,STORES,OWN,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1116,PRIOR_PAYMENTS_DELAYED,APPLIANCES,3960,8,4,F,NONE,3,SAVINGS_INSURANCE,46,NONE,OWN,2,2,1,0,0
996,720,ALL_CREDITS_PAID_BACK,CAR_NEW,1834,7,3,M,NONE,1,SAVINGS_INSURANCE,27,NONE,OWN,1,3,1,0,1
997,558,CREDITS_PAID_TO_DATE,CAR_NEW,2348,1,3,M,CO-APPLICANT,2,SAVINGS_INSURANCE,42,STORES,OWN,1,1,1,0,1
998,1170,PRIOR_PAYMENTS_DELAYED,FURNITURE,5457,4,3,M,CO-APPLICANT,3,CAR_OTHER,42,NONE,OWN,1,3,1,1,1


#### Analisys



In [ ]:
df["OTHERS_ON_LOAN"].value_counts()

NONE            1855
CO-APPLICANT     331
GUARANTOR         51
Name: OTHERS_ON_LOAN, dtype: int64

In [ ]:
df[["ALLOW", "OTHERS_ON_LOAN"]].groupby("OTHERS_ON_LOAN").sum()

,ALLOW
OTHERS_ON_LOAN,
CO-APPLICANT,108
GUARANTOR,13
NONE,1345


In [ ]:
df["FOREIGN_WORKER"].value_counts()

1.0    2183
0.0      54
Name: FOREIGN_WORKER, dtype: int64

In [ ]:
df[["ALLOW", "FOREIGN_WORKER"]].groupby("FOREIGN_WORKER").sum()

,ALLOW
FOREIGN_WORKER,
0.0,25
1.0,1441


In [ ]:
df["DEPENDENTS"].value_counts()

1.0    1873
2.0     364
Name: DEPENDENTS, dtype: int64

In [ ]:
df[["ALLOW", "DEPENDENTS"]].groupby("DEPENDENTS").sum()

,ALLOW
DEPENDENTS,
1.0,1316
2.0,150


### Modeling

In [ ]:
df.select_dtypes("object")

,CHECKING_BALANCE,CREDIT_HISTORY,EXISTING_SAVINGS,SEX,PROPERTY,HOUSING,INSTALLMENT_PLANS,LOAN_PURPOSE,OTHERS_ON_LOAN
ID,,,,,,,,,
98,32.83,CREDITS_PAID_TO_DATE,476.7,F,SAVINGS_INSURANCE,OWN,NONE,OTHER,NONE
317,-150.05,CREDITS_PAID_TO_DATE,49.04,F,SAVINGS_INSURANCE,OWN,STORES,CAR_NEW,NONE
407,-185.04,PRIOR_PAYMENTS_DELAYED,49.65,M,REAL_ESTATE,OWN,NONE,FURNITURE,NONE
2893,NO_CHECKING,OUTSTANDING_CREDIT,695.07,M,UNKNOWN,FREE,NONE,APPLIANCES,CO-APPLICANT
3412,-51.41,ALL_CREDITS_PAID_BACK,71.68,F,REAL_ESTATE,RENT,NONE,CAR_NEW,NONE
...,...,...,...,...,...,...,...,...,...
1317323,-1814.54,NO_CREDITS,83.97,F,SAVINGS_INSURANCE,OWN,NONE,FURNITURE,NONE
1318378,128.65,CREDITS_PAID_TO_DATE,391.93,M,CAR_OTHER,OWN,NONE,CAR_USED,NONE
1319170,9641.84,CREDITS_PAID_TO_DATE,72.52,M,SAVINGS_INSURANCE,OWN,NONE,FURNITURE,NONE


In [ ]:
#features = ["AGE","TELEPHONE","EMPLOYMENT_DURATION","PROPERTY","CURRENT_RESIDENCE_DURATION","PAYMENT_TERM","INSTALLMENT_PERCENT",]

X = df.drop(["CHECKING_BALANCE", "EXISTING_SAVINGS", "ALLOW"], axis=1)
y = df["ALLOW"]
X

,CREDIT_HISTORY,EXISTING_CREDITS_COUNT,SEX,AGE,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,EMPLOYMENT_DURATION,PROPERTY,HOUSING,CURRENT_RESIDENCE_DURATION,PAYMENT_TERM,INSTALLMENT_PLANS,INSTALLMENT_PERCENT,LOAN_PURPOSE,LOAN_AMOUNT,OTHERS_ON_LOAN
ID,,,,,,,,,,,,,,,,,,
98,CREDITS_PAID_TO_DATE,1.0,F,32.0,3.0,1.0,0.0,1.0,0.0,SAVINGS_INSURANCE,OWN,3.0,961.0,NONE,3.0,OTHER,1889.0,NONE
317,CREDITS_PAID_TO_DATE,2.0,F,37.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,2.0,540.0,STORES,2.0,CAR_NEW,462.0,NONE
407,PRIOR_PAYMENTS_DELAYED,2.0,M,28.0,3.0,1.0,1.0,0.0,3.0,REAL_ESTATE,OWN,3.0,465.0,NONE,2.0,FURNITURE,250.0,NONE
2893,OUTSTANDING_CREDIT,2.0,M,49.0,3.0,2.0,1.0,1.0,10.0,UNKNOWN,FREE,4.0,1050.0,NONE,5.0,APPLIANCES,7138.0,CO-APPLICANT
3412,ALL_CREDITS_PAID_BACK,1.0,F,19.0,2.0,1.0,0.0,1.0,0.0,REAL_ESTATE,RENT,2.0,155.0,NONE,2.0,CAR_NEW,1523.0,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317323,NO_CREDITS,1.0,F,32.0,3.0,1.0,0.0,1.0,3.0,SAVINGS_INSURANCE,OWN,2.0,660.0,NONE,3.0,FURNITURE,1822.0,NONE
1318378,CREDITS_PAID_TO_DATE,2.0,M,43.0,3.0,1.0,0.0,1.0,3.0,CAR_OTHER,OWN,3.0,450.0,NONE,3.0,CAR_USED,4097.0,NONE
1319170,CREDITS_PAID_TO_DATE,1.0,M,40.0,3.0,1.0,0.0,1.0,1.0,SAVINGS_INSURANCE,OWN,4.0,810.0,NONE,3.0,FURNITURE,4650.0,NONE


In [ ]:
# c_property = np.array(df["PROPERTY"]).reshape(-1,1)
# c_property = OrdinalEncoder().fit_transform(c_property)
# X["PROPERTY"] = c_property

# X = OrdinalEncoder().fit_transform(X)

objs = np.array(df.select_dtypes("object"))
objs = OrdinalEncoder().fit_transform(objs)
X[df.select_dtypes("object").columns] = objs
X

,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,CHECKING_BALANCE,EXISTING_SAVINGS
ID,,,,,,,,,,,,,,,,,,,,
98,CREDITS_PAID_TO_DATE,1.0,6.0,32.0,3.0,1.0,0.0,2.0,0.0,2.0,OWN,1.0,1.0,NONE,3.0,OTHER,1889.0,NONE,991.0,1068.0
317,CREDITS_PAID_TO_DATE,1.0,2.0,37.0,3.0,1.0,0.0,2.0,1.0,2.0,OWN,2.0,1.0,STORES,2.0,CAR_NEW,462.0,NONE,138.0,1088.0
407,PRIOR_PAYMENTS_DELAYED,4.0,5.0,28.0,3.0,1.0,1.0,2.0,3.0,1.0,OWN,1.0,1.0,NONE,2.0,FURNITURE,250.0,NONE,205.0,1094.0
2893,OUTSTANDING_CREDIT,3.0,0.0,49.0,3.0,2.0,1.0,0.0,10.0,3.0,FREE,1.0,0.0,NONE,5.0,APPLIANCES,7138.0,CO-APPLICANT,1262.0,1466.0
3412,ALL_CREDITS_PAID_BACK,0.0,2.0,19.0,2.0,1.0,0.0,2.0,0.0,1.0,RENT,1.0,2.0,NONE,2.0,CAR_NEW,1523.0,NONE,439.0,1499.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317323,NO_CREDITS,2.0,5.0,32.0,3.0,1.0,0.0,2.0,3.0,2.0,OWN,1.0,1.0,NONE,3.0,FURNITURE,1822.0,NONE,201.0,1731.0
1318378,CREDITS_PAID_TO_DATE,1.0,3.0,43.0,3.0,1.0,1.0,2.0,3.0,0.0,OWN,1.0,1.0,NONE,3.0,CAR_USED,4097.0,NONE,727.0,873.0
1319170,CREDITS_PAID_TO_DATE,1.0,5.0,40.0,3.0,1.0,1.0,2.0,1.0,2.0,OWN,1.0,1.0,NONE,3.0,FURNITURE,4650.0,NONE,1252.0,1521.0


In [ ]:
test_pct = 0.3 # Separaremos 30% dos dados para testes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_pct, random_state=42)

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo (número de predições assertivas sobre número total de testes): {acc}")

ValueError: ignored

In [ ]:
tpot = TPOTClassifier(generations=10, population_size=100, verbosity=2, 
                      scoring="f1", random_state=42, early_stop=3)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

Optimization Progress:   0%|          | 0/1100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8503646683321652

Generation 2 - Current best internal CV score: 0.8503646683321652

Generation 3 - Current best internal CV score: 0.8503646683321652

Generation 4 - Current best internal CV score: 0.8527818824071144

Generation 5 - Current best internal CV score: 0.8527818824071144

Generation 6 - Current best internal CV score: 0.8527818824071144

Generation 7 - Current best internal CV score: 0.8527818824071144

Generation 8 - Current best internal CV score: 0.8544134842614126

Generation 9 - Current best internal CV score: 0.8544134842614126

Generation 10 - Current best internal CV score: 0.8544134842614126

Best pipeline: GradientBoostingClassifier(LogisticRegression(input_matrix, C=20.0, dual=False, penalty=l2), learning_rate=0.01, max_depth=5, max_features=0.25, min_samples_leaf=11, min_samples_split=20, n_estimators=100, subsample=1.0)
0.8677685950413223


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC

# features = ["AGE","TELEPHONE","EMPLOYMENT_DURATION","PROPERTY","CURRENT_RESIDENCE_DURATION","PAYMENT_TERM","INSTALLMENT_PERCENT",]
# with all enconded
# Average CV score on the training set was: 0.8399532793524223
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LinearSVC(C=0.001, dual=True, loss="hinge", penalty="l2", tol=1e-05)),
    ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.1, min_samples_leaf=18, min_samples_split=9, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print(f1_score(y_test, results))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  "X does not have valid feature names, but"


0.8595213319458898


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
# features = ["AGE","TELEPHONE","EMPLOYMENT_DURATION","PROPERTY","CURRENT_RESIDENCE_DURATION","PAYMENT_TERM","INSTALLMENT_PERCENT",]
# with PROPERTY encoded
# Average CV score on the training set was: 0.858908341915551
exported_pipeline = ExtraTreesClassifier(
    bootstrap=False, criterion="entropy", max_features=0.2, 
    min_samples_leaf=6, min_samples_split=9, n_estimators=100, random_state=42
)
exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print(f1_score(y_test, results))

0.858908341915551


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


## Selecting Tis model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Average CV score on the training set was: 0.8544134842614126
exported_pipeline = make_pipeline(
    # StackingEstimator(estimator=LogisticRegression(C=20.0, dual=False, penalty="l2")),
    GradientBoostingClassifier(learning_rate=0.01, max_depth=5, max_features=0.25, min_samples_leaf=11, min_samples_split=20, n_estimators=100, subsample=1.0, random_state=42)
)
exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print(f1_score(y_test, results))

0.8788501026694046
